# From Prediction to Persuasion

## Precision-targeted Marketing with Amazon SageMaker AI Canvas and Amazon Nova Generative Foundation Models

__Description:__ This script is used to invoke the deployed Amazon SageMaker endpoint. The endpoint is used to predict the propensity of a customer to buy electronic products during the Christmas season - True or False. If True, a targeted marketing piece is generated based on the same inference data using Amazon Nova Pro.

__Inputs (features):__ ["age", "gender", "race", "married", "salary", "tech_savvy_score", "price_sensitivity_score", "last_item_purchased", "previous_item_purchased"]

__Output (prediction):__ ["buy"]

__Author:__ Gary A. Stafford

__Date:__ 2024-12-06

## Amazon SageMaker Real-time Inference Endpoint

Invoke the Canvas model for a buy prediction.

In [ ]:
! pip install boto3 botocore numpy pandas -Uq

In [ ]:
import csv
from dataclasses import dataclass
import json
from typing import List, Tuple

import boto3
from botocore.exceptions import ClientError
from IPython.display import display, HTML
import numpy as np
import pandas as pd

# Create boto3 clients
sm_client = boto3.client("sagemaker")
sm_runtime_client = boto3.client("runtime.sagemaker")

In [ ]:
# List all in-service endpoints
response = sm_client.list_endpoints(SortBy="Name")
for endpoint in response["Endpoints"]:
    if endpoint["EndpointStatus"] == "InService":
        print(endpoint["EndpointName"])

In [ ]:
# Update to your model's endpoint name
ENDPOINT_NAME = "canvas-ptb-xmas-electronics-v8b-model"

In [ ]:
def invoke_endpoint(values) -> str:
    """
    Invokes an endpoint with the given values and returns the response.

    Args:
        values (list): A list of values to be sent to the endpoint.

    Returns:
        str: The response from the endpoint.

    Raises:
        botocore.exceptions.ClientError: If the request to the endpoint fails.
    """

    body = (
        pd.DataFrame(
            [values],
        )
        .to_csv(header=False, index=False)
        .encode("utf-8")
    )

    try:
        response = sm_runtime_client.invoke_endpoint(
            EndpointName=ENDPOINT_NAME,
            ContentType="text/csv",
            Body=body,
            Accept="application/json",
        )
        return response
    except ClientError as error:
        message = error.response["Error"]["Message"]
        print("A client error occurred: " + format(message))
        return None

In [ ]:
# Sample customer data
target_customers = [
    {
        "customer_id": "d1b7c683-dc24-4345-9a7c-ef981c7d8020",
        "first_name": "Bruno",
        "phone": "111-222-3333",
        "email": "bruno7867@example.com",
        "age": 21,
        "gender": "Male",
        "race": "White",
        "married": False,
        "salary": 108_000,
        "tech_savvy_score": 0.78,
        "price_sensitivity_score": 0.34,
        "previous_item_purchased": "Game console and accessories",
        "last_item_purchased": "Computer, monitor, and accessories",
    },
    {
        "customer_id": "22444d4d-f013-475d-be49-b29b6a828623",
        "first_name": "Mira",
        "phone": "111-222-3333",
        "email": "mira3994@example.com",
        "age": 62,
        "gender": "Female",
        "race": "Asian",
        "married": True,
        "salary": 36000,
        "tech_savvy_score": 0.15,
        "price_sensitivity_score": -1.12,
        "previous_item_purchased": "Smartphone and accessories",
        "last_item_purchased": "Television and accessories",
    },
    {
        "customer_id": "1827e17f-0415-4c4b-9866-17f5fd1ffc97",
        "first_name": "Eduardo",
        "phone": "111-222-3333",
        "email": "eduardo3730@example.com",
        "age": 53,
        "gender": "Male",
        "race": "Hispanic",
        "married": True,
        "salary": 118000,
        "tech_savvy_score": 1.99,
        "price_sensitivity_score": -2.04,
        "previous_item_purchased": "Smartphone and accessories",
        "last_item_purchased": "Computer, monitor, and accessories",
    },
    {
        "customer_id": "40837f25-5959-466b-9b87-6ffa7e0f3169",
        "first_name": "Mary",
        "phone": "111-222-3333",
        "email": "mary9898@example.com",
        "age": 41,
        "gender": "Female",
        "race": "White",
        "married": True,
        "salary": 92000,
        "tech_savvy_score": 2.45,
        "price_sensitivity_score": -1.48,
        "previous_item_purchased": "Television and accessories",
        "last_item_purchased": "Television and accessories",
    },
    {
        "customer_id": "88107af8-d521-44e8-8d69-e5c46aa054fa",
        "first_name": "Akito",
        "phone": "111-222-3333",
        "email": "akito9458@example.com",
        "age": 49,
        "gender": "Male",
        "race": "Asian",
        "married": False,
        "salary": 125000,
        "tech_savvy_score": 0.11,
        "price_sensitivity_score": 0.6,
        "previous_item_purchased": "Computer, monitor, and accessories",
        "last_item_purchased": "Television and accessories",
    },
]

In [ ]:
# Single prediction
target_customer = target_customers[0]

inference_values = [
    target_customer["age"],
    target_customer["gender"],
    target_customer["race"],
    target_customer["married"],
    target_customer["salary"],
    target_customer["tech_savvy_score"],
    target_customer["price_sensitivity_score"],
    target_customer["last_item_purchased"],
    target_customer["previous_item_purchased"],
]

response = invoke_endpoint(inference_values)
body_json = json.loads(response["Body"].read().decode("utf-8"))
print(json.dumps(body_json, indent=4))

predicted_label = body_json["predictions"][0]["predicted_label"]
print(f"Prediction: {predicted_label}")

In [ ]:
# Multiple predictions
for target_customer in target_customers:
    inference_values = [
        target_customer["age"],
        target_customer["gender"],
        target_customer["race"],
        target_customer["married"],
        target_customer["salary"],
        target_customer["tech_savvy_score"],
        target_customer["price_sensitivity_score"],
        target_customer["last_item_purchased"],
        target_customer["previous_item_purchased"],
    ]

    response = invoke_endpoint(inference_values)
    body_json = json.loads(response["Body"].read().decode("utf-8"))

    # classification model - prediction
    predicted_label = body_json["predictions"][0]["predicted_label"]
    print(f"{target_customer['first_name']}: {predicted_label}")

## Amazon Bedrock

Invoke Amazon Nova Pro through Amazon Bedrock.

In [ ]:
bedrock_runtime_client = boto3.client("bedrock-runtime")

MODEL_ID = "amazon.olympus-pro-v1:0"

product_offer = {
    "product": "Apple Watch Series 10",
    "price": "starting at $329",
    "cta_link": "https://bit.ly/4fVvy4V",
    "category": "Smartphone and accessories",
}

SYSTEM_PROMPT = (
    "You are an accomplished copywriter for a leading digital marketing agency."
)

PROMPT_TEMPLATE_EMAIL = f"""Below is information about a target customer in JSON format. 
They have a high propensity to buy electronics during the Christmas Holiday. 
Use the information to write a brief and compelling personalized email message advertising the new {product_offer['product']}, now on sale, {product_offer['price']}.

{target_customer}

Important:
1. The customer's name is {target_customer['first_name']}
2. The company's name is Acme Electronics
3. The call-to-action is 'Shop Now' with the link {product_offer['cta_link']}
4. Don't mention the customer's age, race, or gender
5. Don't include any preamble or wrap the output in extra elements
6. Use emojis where appropriate
7. Output the message in well-formatted HTML, including <head>, <body>, CSS, <style>, <div>, and class elements
8. Include the phrase 'If you wish to unsubscribe, please click Unsubscribe.' at the end of the email
9. Include the phrase '© 2024 Acme Electronics. All rights reserved.' at the end of the email
"""

PROMPT_TEMPLATE_SMS = f"""Below is information about a target customer in JSON format. 
They have a high propensity to buy electronics during the Christmas Holiday. 
Use the information to write a brief and compelling personalized SMS text message advertising the new {product_offer['product']}, now on sale, {product_offer['price']}.

{target_customer}

Important:
1. The customer's name is {target_customer['first_name']}
2. The company's name is Acme Electronics
3. The call-to-action is 'Shop Now' with the link {product_offer['cta_link']}
4. Don't mention the customer's age, race, or gender
5. Don't include any preamble or wrap the output in extra elements
6. Use emojis where appropriate
7. Output the message as text
8. Include the phrase 'Reply STOP to opt out' at the end of the message
"""

In [ ]:
# Preview the final prompts
print(f"Email prompt:\n{PROMPT_TEMPLATE_EMAIL}")
print("\n---\n")
print(f"SMS prompt:\n{PROMPT_TEMPLATE_SMS}")

In [ ]:
if (
    predicted_label
    and target_customer["last_item_purchased"]
    != product_offer["category"]  # optional filter criteria
):
    response = bedrock_runtime_client.converse(
        modelId=MODEL_ID,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "text": PROMPT_TEMPLATE_EMAIL,
                    },
                ],
            },
        ],
        system=[
            {
                "text": SYSTEM_PROMPT,
            },
        ],
        inferenceConfig={
            "maxTokens": 1024,
            "temperature": 0.4,
            "topP": 0.9,
            "stopSequences": [],
        },
    )

In [ ]:
print(json.dumps(response, indent=4))

In [ ]:
message = response["output"]["message"]["content"][0]["text"]

In [ ]:
print(message)

In [ ]:
# For emails in HTML
display(HTML(message))